In [ ]:
# ! pip install pygame

     |████████████████████████████████| 14.0 MB 4.4 MB/s eta 0:00:01


In [9]:
import pygame
import random
import sys

# Basic colors and settings
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 200, 0)
RED = (200, 0, 0)
GRAY = (150, 150, 150)
BLUE = (0, 100, 255)
CELL_SIZE = 30
FONT_NAME = "arial"


pygame.init()
pygame.mixer.init()
pygame.mixer.music.load("Peaceful Oasis (cdef6aa6cf2f4941a19c48303fa7c8e8).mp3")
pygame.mixer.music.play(-1)
font = pygame.font.SysFont(FONT_NAME, 30)
screen = pygame.display.set_mode((640, 480))
pygame.display.set_caption("Laberinto!")
clock = pygame.time.Clock()

victory_img = pygame.image.load("corazon.png").convert_alpha()
logo_img = pygame.image.load("cara_male_editada.png").convert_alpha()
exit_img = pygame.image.load("pokeball.png").convert_alpha()


exit_img = pygame.transform.scale(exit_img, (CELL_SIZE, CELL_SIZE))
# ---------------- Maze Logic ---------------- #
class MazeGame:
    def __init__(self, width, height):
        self.cols = width if width % 2 == 1 else width + 1
        self.rows = height if height % 2 == 1 else height + 1
        self.maze = [['#'] * self.cols for _ in range(self.rows)]
        self.start = (1, 1)
        self.exit = (self.rows - 2, self.cols - 2)
        self.player = list(self.start)
        self.score = 0
        self.coins = set()
        self.generate_maze()
        self.spawn_coins(10)

    def generate_maze(self):
        def carve(x, y):
            directions = [(0, 2), (2, 0), (0, -2), (-2, 0)]
            random.shuffle(directions)
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 < nx < self.rows and 0 < ny < self.cols and self.maze[nx][ny] == '#':
                    self.maze[nx - dx // 2][ny - dy // 2] = ' '
                    self.maze[nx][ny] = ' '
                    carve(nx, ny)

        self.maze[self.start[0]][self.start[1]] = ' '
        carve(*self.start)
        self.maze[self.exit[0]][self.exit[1]] = 'E'

    def spawn_coins(self, amount):
        empty_spaces = [
            (r, c) for r in range(1, self.rows - 1)
            for c in range(1, self.cols - 1)
            if self.maze[r][c] == ' ' and (r, c) not in [self.start, self.exit]
        ]
        self.coins = set(random.sample(empty_spaces, min(amount, len(empty_spaces))))

    def draw(self, surface, player_name,width,height,selected_player):
        surface.fill(BLACK)
        for row in range(self.rows):
            for col in range(self.cols):
                rect = pygame.Rect(col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                if (row, col) == tuple(self.player):
                    # pygame.draw.rect(surface, GREEN, rect)
                    pygame.draw.rect(surface, GRAY, rect)
                    screen.blit(selected_player, (col * CELL_SIZE,row * CELL_SIZE))
                elif (row, col) == self.exit:
                    pygame.draw.rect(surface, GRAY, rect)
                    screen.blit(exit_img, (col * CELL_SIZE,row * CELL_SIZE))
                elif (row, col) in self.coins:
                    pygame.draw.circle(surface, (255, 215, 0), rect.center, CELL_SIZE // 3)
                elif self.maze[row][col] == '#':
                    pygame.draw.rect(surface, WHITE, rect)
                else:
                    pygame.draw.rect(surface, GRAY, rect)

        # Dibujar nombre y puntuación
        score_text = f"{player_name}: {self.score}"
        text_surface = font.render(score_text, True, WHITE)
        surface.blit(text_surface, (width, height))

    def move_player(self, dx, dy):
        new_y = self.player[0] + dy
        new_x = self.player[1] + dx
        if self.maze[new_y][new_x] != '#':
            self.player = [new_y, new_x]
            if tuple(self.player) in self.coins:
                self.coins.remove(tuple(self.player))
                self.score += 100


# ---------------- UI Helpers ---------------- #
def draw_text(text, size, color, center):
    font_obj = pygame.font.SysFont(FONT_NAME, size)
    text_surface = font_obj.render(text, True, color)
    rect = text_surface.get_rect(center=center)
    screen.blit(text_surface, rect)

def draw_button(rect, text):
    pygame.draw.rect(screen, BLUE, rect)
    pygame.draw.rect(screen, WHITE, rect, 2)
    draw_text(text, 24, WHITE, rect.center)

def wait_for_key():
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

# ---------------- Game States ---------------- #
def start_screen():
    global screen
    screen = pygame.display.set_mode((640, 480))
    name = ""
    active = False
    input_box = pygame.Rect(20, 120, 200, 36)

    while True:
        screen.fill(WHITE)
        draw_text("Laberinto de male", 48, BLACK, (320, 60))
        draw_text("Nombre:", 24, BLACK, (65, 100))

        pygame.draw.rect(screen, BLACK, input_box, 2)

        
        img = pygame.transform.smoothscale(logo_img, (400, 400))
        img_rect = img.get_rect(center=(420, 300))
        screen.blit(img, img_rect)
        
        name_to_show = "Escribe aquí..."
        if active:
            name_to_show = name


        draw_text(name_to_show, 24, BLACK, input_box.center)

        btn_small = pygame.Rect(20, 180, 200, 40)
        btn_medium = pygame.Rect(20, 230, 200, 40)
        btn_large = pygame.Rect(20, 280, 200, 40)
        btn_exit = pygame.Rect(20, 340, 200, 40)

        draw_button(btn_small, "Pequeño (15x15)")
        draw_button(btn_medium, "Mediano (21x21)")
        draw_button(btn_large, "Grande (31x31)")
        draw_button(btn_exit, "Salir")

        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if input_box.collidepoint(event.pos):
                    active = True  
                    pygame.draw.rect(screen, WHITE, input_box, 2)
                else:
                    active = False
                if btn_small.collidepoint(event.pos) and name:
                    return (15, 15, name)
                elif btn_medium.collidepoint(event.pos) and name:
                    return (21, 21, name)
                elif btn_large.collidepoint(event.pos) and name:
                    return (31, 31, name)
                elif btn_exit.collidepoint(event.pos):
                    pygame.quit()
                    sys.exit()
            elif event.type == pygame.KEYDOWN and active:
                if event.key == pygame.K_RETURN:
                    active = False
                elif event.key == pygame.K_BACKSPACE:
                    name = name[:-1]
                else:
                    if len(name) < 12:
                        name += event.unicode

        clock.tick(30)

def select_character_screen():
    global screen
    screen = pygame.display.set_mode((640, 480))
    selected_image = None

    # Load and scale player images
    characters_selection = []
    characters_players = []
    for i in range(1, 4):
        img = pygame.image.load(f"player_{i}.png").convert_alpha()
        img = pygame.transform.scale(img, (CELL_SIZE*5, CELL_SIZE*5))
        img_player = pygame.transform.scale(img, (CELL_SIZE, CELL_SIZE))
        characters_selection.append(img)
        characters_players.append(img_player)

    # Set positions
    positions = [
        (160, 200),
        (320, 200),
        (480, 200)
    ]

    while True:
        screen.fill(BLACK)
        draw_text("Elegí tu personaje", 36, WHITE, (320, 80))

        # Draw images
        for i, img in enumerate(characters_selection):
            rect = img.get_rect(center=positions[i])
            screen.blit(img, rect)

        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        
            elif event.type == pygame.MOUSEBUTTONDOWN:
                for i, pos in enumerate(positions):
                    rect = characters_selection[i].get_rect(center=pos)
                    if rect.collidepoint(event.pos):                        
                        return characters_players[i]  # Return the selected image
        
        clock.tick(60)


def win_screen():
    global screen
    screen = pygame.display.set_mode((640, 480))

    # Animación: variables
    scale = 1.0
    direction = 1
    max_scale = 1.5
    min_scale = 0.8
    scale_speed = 0.01

    while True:
        screen.fill(BLACK)
        draw_text(" ESCAPASTE! GENIAA! ", 36, GREEN, (320, 60))

        # Animar la imagen (agrandar y achicar)
        scaled_size = int(128 * scale)
        img = pygame.transform.smoothscale(victory_img, (scaled_size, scaled_size))
        img_rect = img.get_rect(center=(320, 200))
        screen.blit(img, img_rect)

        # Botón volver
        btn_back = pygame.Rect(220, 360, 200, 50)
        draw_button(btn_back, "Volver al Menu")

        pygame.display.flip()

        # Lógica de animación
        scale += scale_speed * direction
        if scale >= max_scale or scale <= min_scale:
            direction *= -1

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if btn_back.collidepoint(event.pos):
                    return

        clock.tick(60)

def game_loop(width, height, player_name,selected_player):
    global screen
    window_width = width * CELL_SIZE
    window_height = height * CELL_SIZE + 40  # espacio para el texto arriba
    screen = pygame.display.set_mode((window_width, window_height))

    game = MazeGame(width, height)
    win = False
    while not win:
        game.draw(screen, player_name,0,window_height - 40,selected_player)
        pygame.display.flip()
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    game.move_player(0, -1)
                elif event.key == pygame.K_DOWN:
                    game.move_player(0, 1)
                elif event.key == pygame.K_LEFT:
                    game.move_player(-1, 0)
                elif event.key == pygame.K_RIGHT:
                    game.move_player(1, 0)

        if tuple(game.player) == game.exit:
            win = True
    win_screen()


# ---------------- Main Loop ---------------- #
def main():
    while True:
        width, height, player_name = start_screen()
        selected_player = select_character_screen()
        game_loop(width, height, player_name,selected_player)


if __name__ == "__main__":
    main()


SystemExit: 